In [4]:
import time
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime

In [5]:
headers = {"X-Riot-Token": "API TOKEN HERE"}

In [6]:
response = requests.get("https://na1.api.riotgames.com/tft/league/v1/challenger", headers=headers)

In [7]:
challenger_dict = response.json()

challenger_dict["entries"]

[{'summonerId': 'tvBRD5oEaz13VvCMHisjij4bt71p2L-wCbSEv4px8fgb1Rg',
  'summonerName': 'EAOST',
  'leaguePoints': 321,
  'rank': 'I',
  'wins': 47,
  'losses': 246,
  'veteran': False,
  'inactive': False,
  'freshBlood': True,
  'hotStreak': False},
 {'summonerId': '7ED4H7fgBl5t4hD5PALP6oYDWnW2mF9SqUeFm5DYvhl7-ns',
  'summonerName': 'Aesah',
  'leaguePoints': 325,
  'rank': 'I',
  'wins': 80,
  'losses': 306,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'summonerId': '1LofYRaPHU7ln34Qc5e7BJ-FRVLLgEl3kQSU_4wKM0Ta8t0',
  'summonerName': 'bertasaurus',
  'leaguePoints': 529,
  'rank': 'I',
  'wins': 68,
  'losses': 272,
  'veteran': True,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False},
 {'summonerId': 'xYXf7ODcecxX7IpqPFzUi1iTkmABIwmIFJf4rCQEBa8QnQU',
  'summonerName': 'TheMagykal',
  'leaguePoints': 295,
  'rank': 'I',
  'wins': 65,
  'losses': 299,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': 

In [8]:
i = 0
challenger_sumnam = []
challenger_lp = []
challenger_wins = []
challenger_losses = []

while i < len(challenger_dict["entries"]):
    challenger_sumnam.append(challenger_dict["entries"][i]["summonerName"])
    challenger_lp.append(challenger_dict["entries"][i]["leaguePoints"])
    challenger_wins.append(challenger_dict["entries"][i]["wins"])
    challenger_losses.append(challenger_dict["entries"][i]["losses"])
    i += 1

In [9]:
challenger_df = pd.DataFrame(columns = ["Summoner_Name","LP","Wins","Losses"])

In [10]:
challenger_df["Summoner_Name"] = challenger_sumnam
challenger_df["LP"] = challenger_lp
challenger_df["Wins"] = challenger_wins
challenger_df["Losses"] = challenger_losses
challenger_df["Win_Rate"] = challenger_df["Wins"]/(challenger_df["Wins"]+challenger_df["Losses"])

In [11]:
challenger_df["Win_Rate_2"] = [f'{round(wr*100, 2)}%' for wr in challenger_df["Win_Rate"]]

In [12]:
challenger_df.drop(["Win_Rate"],axis=1, inplace= True)

In [13]:
challenger_df.sort_values(["LP"],ascending=False,inplace= True)
challenger_df.rename(columns={"Win Rate 2":"Win Rate"}, inplace = True)
challenger_df.reset_index(inplace=True)
challenger_df.drop(["index"],axis=1 ,inplace= True)

In [ ]:
#challenger_df.set_index(["Summoner_Name"],inplace=True)

In [14]:
#challenger_df.to_json("./challenger_info.json")
challenger_df.reset_index()

,index,Summoner_Name,LP,Wins,Losses,Win_Rate_2
0,0,GrandVice8,1334,103,363,22.1%
1,1,Hiraino,1136,63,310,16.89%
2,2,DeliciousMilkGG,1109,74,177,29.48%
3,3,Silliest,1093,76,192,28.36%
4,4,Krmx,1090,80,284,21.98%
...,...,...,...,...,...,...
194,194,best Taro NA,111,39,194,16.74%
195,195,J or C,110,32,105,23.36%
196,196,Doril,90,88,542,13.97%
197,197,vx daodao46,81,101,350,22.39%


In [15]:
lp_lst = challenger_df.set_index("Summoner_Name")["LP"].to_dict()
win_lst = challenger_df.set_index("Summoner_Name")["Wins"].to_dict()
loss_lst = challenger_df.set_index("Summoner_Name")["Losses"].to_dict()
win_rate_lst = challenger_df.set_index("Summoner_Name")["Win_Rate_2"].to_dict()

In [16]:
summoner_data = []
for key in lp_lst:
    temp = {
        "Summoner_Name": key,
        "LP": lp_lst[key],
        "Wins": win_lst[key],
        "Losses": loss_lst[key],
        "Win_Rate": win_rate_lst[key]
    }
    
    summoner_data.append(temp)

In [17]:
import json

In [18]:
with open("challenger_info.json", 'w') as fp:
  json.dump(summoner_data, fp)

In [53]:
player_data = pd.read_csv("player_data.csv")

In [61]:
player_data.drop(["Unnamed: 0"],axis=1,inplace=True)

In [84]:
player_data_arr = []
#for key in 

player_lp_lst = player_data.set_index("summoner_name")["league_points"].to_dict()
player_uid_lst = player_data.set_index("summoner_name")["puuid"].to_dict()

In [85]:
for key in player_lp_lst:
    temp = {
        "Summoner_Name": key,
        "LP:" : player_lp_lst[key],
        "PUUID" : player_uid_lst[key]
    }
    
    player_data_arr.append(temp)

In [88]:
with open("player_data.json","w") as fp:
    json.dump(player_data_arr, fp)

In [224]:
match_data = pd.read_csv("match_ids.csv")
complete_match_data = pd.read_csv("complete_match_data.csv")

match_data.drop(["Unnamed: 0"], axis=1, inplace=True)
complete_match_data.drop(["Unnamed: 0"], axis=1, inplace=True)

In [340]:
master_sheet = []

for i in range(len(match_data.index)):
    for j in range(len(complete_match_data.index)):
        if match_data.iloc[i][0] == complete_match_data.iloc[j][0] \
        and match_data.iloc[i][1] == complete_match_data.iloc[j][1]:
            temp = {}
            temp = {
                "PUUID": match_data.iloc[i][0],
                "Match_ID": match_data.iloc[i][1],
                "Placement": complete_match_data.iloc[j][6],
                "Champions": complete_match_data.iloc[j][-1],
                "Traits": complete_match_data.iloc[j][-2]
            }
            master_sheet.append(temp)
            print(temp)
            break

print(master_sheet)      
len(master_sheet)
        

{'PUUID': 'OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1djhBqxsj360DcaldQDCukF_G9PButnSFg', 'Match_ID': 'NA1_3249805878', 'Placement': 2, 'Champions': "[('Annie', 3, 2, []), ('Zed', 4, 2, [24, 66, 16]), ('Lux', 5, 2, [22, 35, 35]), ('Yorick', 3, 2, []), ('Malphite', 3, 2, [37]), ('Brand', 3, 1, [10201]), ('Nautilus', 2, 2, []), ('Ornn', 0, 1, [10201]), ('', 4, 2, [14, 2])]", 'Traits': "[('Avatar', 1), ('Electric', 3), ('Inferno', 1), ('Light', 0), ('Mage', 0), ('Mountain', 0), ('Ocean', 0), ('Set2_Assassin', 0), ('Summoner', 1), ('Warden', 2)]"}
{'PUUID': 'OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1djhBqxsj360DcaldQDCukF_G9PButnSFg', 'Match_ID': 'NA1_3249791129', 'Placement': 1, 'Champions': "[('Lux', 5, 2, [44, 44, 37]), ('Malphite', 3, 2, [45, 6]), ('Annie', 3, 2, [35]), ('Zed', 4, 2, [99, 19, 57]), ('Qiyana', 2, 2, []), ('Nautilus', 2, 2, []), ('', 4, 1, [78]), ('Thresh', 1, 2, []), ('Yorick', 3, 2, [10202])]", 'Traits': "[('Avatar', 1), ('Electric', 0), ('Inferno', 0), ('Light', 

{'PUUID': 'PY8453ZFDl78q0BN6ZsYU8m4U7E-4Sr2y6YcT5lNeo0th-3AuXRBb9RwGkwkUEO4ongZnqJq7d6WuQ', 'Match_ID': 'NA1_3250236114', 'Placement': 1, 'Champions': "[('Soraka', 2, 2, []), ('Janna', 3, 2, []), ('Azir', 2, 3, [14, 68]), ('Annie', 3, 2, [67, 10204]), ('Singed', 4, 2, [37, 25, 38]), ('', 4, 2, []), ('Yorick', 3, 2, [15, 7, 36]), ('Malphite', 3, 2, [])]", 'Traits': "[('Alchemist', 1), ('Desert', 0), ('Inferno', 1), ('Light', 1), ('Mountain', 0), ('Mystic', 1), ('Poison', 0), ('Summoner', 1), ('Warden', 1), ('Wind', 0)]"}
{'PUUID': 'PY8453ZFDl78q0BN6ZsYU8m4U7E-4Sr2y6YcT5lNeo0th-3AuXRBb9RwGkwkUEO4ongZnqJq7d6WuQ', 'Match_ID': 'NA1_3250233154', 'Placement': 7, 'Champions': "[('Janna', 3, 2, []), ('Zyra', 0, 2, [34, 44, 37]), ('Annie', 3, 2, [35]), ('Nami', 4, 1, [10204]), ('Nautilus', 2, 1, [10204]), ('Yasuo', 1, 1, []), ('Yorick', 3, 2, []), ('', 4, 1, [68])]", 'Traits': "[('Inferno', 1), ('Light', 0), ('Mystic', 1), ('Ocean', 1), ('Set2_Blademaster', 0), ('Summoner', 1), ('Warden', 1), ('

{'PUUID': 'Mtiy5xQq2B08E8UKrWt_AcHemRq40W5xJQKfC6wIONGMCJHXRQ380TPlx115FKjSDG3xWsKBPrk-8g', 'Match_ID': 'NA1_3250292795', 'Placement': 8, 'Champions': "[('Soraka', 2, 2, []), ('', 1, 1, [77, 28]), ('Janna', 3, 1, [35]), ('Taric', 4, 1, []), ('', 4, 1, [3]), ('Malphite', 3, 2, [10201]), ('Lucian', 3, 2, [10201, 28, 2])]", 'Traits': "[('Crystal', 0), ('Inferno', 0), ('Light', 0), ('Mountain', 0), ('Mystic', 1), ('Set2_Blademaster', 1), ('Shadow', 0), ('Soulbound', 1), ('Warden', 1), ('Wind', 0)]"}
{'PUUID': 'Mtiy5xQq2B08E8UKrWt_AcHemRq40W5xJQKfC6wIONGMCJHXRQ380TPlx115FKjSDG3xWsKBPrk-8g', 'Match_ID': 'NA1_3250300260', 'Placement': 7, 'Champions': "[('Varus', 1, 2, []), ('Twitch', 3, 2, [14, 12, 26]), ('Ashe', 3, 1, []), ('DrMundo', 2, 1, [10203]), ('Ezreal', 2, 2, [57, 10203]), ('Malphite', 3, 2, [55]), ('Taric', 4, 1, [79]), ('Singed', 4, 1, [])]", 'Traits': "[('Alchemist', 1), ('Berserker', 0), ('Crystal', 1), ('Inferno', 0), ('Mountain', 0), ('Poison', 1), ('Set2_Glacial', 0), ('Set2_R

{'PUUID': 'VJQMuXGE5493waSe6PFxLrn2zPrQfMAi7ZDTBMwU7rXt3rlGuaqkTRd-RD2eujFNxpE1LBNjYGZ3ug', 'Match_ID': 'NA1_3249408423', 'Placement': 2, 'Champions': "[('Ashe', 3, 1, []), ('Janna', 3, 2, [34]), ('Brand', 3, 2, [67, 25, 33]), ('Annie', 3, 2, [10204]), ('Taric', 4, 1, [67]), ('', 4, 1, [37, 44, 39]), ('Qiyana', 2, 2, []), ('Thresh', 1, 2, [10204]), ('Nautilus', 2, 2, [])]", 'Traits': "[('Crystal', 1), ('Inferno', 1), ('Mage', 0), ('Mystic', 0), ('Ocean', 1), ('Set2_Assassin', 0), ('Set2_Ranger', 0), ('Summoner', 0), ('Warden', 2), ('Wind', 1)]"}
{'PUUID': 'vufERsZYvVzZKQzIUtxTyChDrvHePm6UjkOn1FmYTgSU1irJoAM041mpOqtURo15b9mB8P7oJm6owA', 'Match_ID': 'NA1_3250484468', 'Placement': 5, 'Champions': "[('Volibear', 1, 2, [10202]), ('Renekton', 0, 3, [78, 3]), ('DrMundo', 2, 2, [33, 35]), ('Sion', 2, 2, []), ('Olaf', 3, 2, [10202, 46, 46]), ('Jax', 1, 2, []), ('Khazix', 3, 2, [46, 19])]", 'Traits': "[('Berserker', 2), ('Desert', 1), ('Electric', 0), ('Light', 0), ('Poison', 0), ('Set2_Assassin

{'PUUID': '9Hido7hDNW_eDisdXh5TAkyNvU1jIo_rIddx2pfH5ewKH9ICSpn9a9xI5XZqrO1f_dGlnp0QwQiH3A', 'Match_ID': 'NA1_3249511819', 'Placement': 4, 'Champions': "[('Janna', 3, 1, []), ('Azir', 2, 2, [99]), ('Zyra', 0, 2, [35, 44, 37]), ('Brand', 3, 2, []), ('MasterYi', 4, 1, []), ('Qiyana', 2, 2, [14, 10204]), ('Annie', 3, 2, [10204, 9, 79]), ('Nautilus', 2, 2, []), ('Thresh', 1, 2, [])]", 'Traits': "[('Desert', 0), ('Inferno', 1), ('Mage', 0), ('Mystic', 1), ('Ocean', 1), ('Set2_Assassin', 0), ('Set2_Blademaster', 0), ('Shadow', 0), ('Summoner', 1), ('Warden', 1), ('Wind', 1)]"}
{'PUUID': '9Hido7hDNW_eDisdXh5TAkyNvU1jIo_rIddx2pfH5ewKH9ICSpn9a9xI5XZqrO1f_dGlnp0QwQiH3A', 'Match_ID': 'NA1_3249482298', 'Placement': 2, 'Champions': "[('Varus', 1, 1, [46]), ('Janna', 3, 1, []), ('Kindred', 2, 3, [19, 22, 44]), ('Nami', 4, 2, [10204, 79, 66]), ('Sion', 2, 2, []), ('Sion', 2, 2, [10204]), ('Malzahar', 1, 2, [35]), ('', 4, 1, [99]), ('Thresh', 1, 1, [])]", 'Traits': "[('Berserker', 0), ('Inferno', 1), (

...

{'PUUID': 'bFQ1v3MkwLuFtOmmU1JgwQu-TE7w2LAa6ihU-zSSoqcXwLQpgeuUk29efs4d4wMWd6Ci63HzZyjy8A', 'Match_ID': 'NA1_3250268172', 'Placement': 5, 'Champions': "[('Zyra', 0, 2, [37]), ('', 4, 1, []), ('Brand', 3, 2, []), ('Qiyana', 2, 2, [10201, 36]), ('Annie', 3, 1, [12]), ('Soraka', 2, 2, []), ('Malzahar', 1, 2, [10201]), ('Kindred', 2, 2, [24, 39, 11]), ('Sion', 2, 2, [])]", 'Traits': "[('Berserker', 0), ('Inferno', 2), ('Light', 0), ('Mage', 0), ('Mystic', 0), ('Set2_Assassin', 0), ('Set2_Ranger', 0), ('Shadow', 1), ('Summoner', 1), ('Warden', 0)]"}
{'PUUID': 'bFQ1v3MkwLuFtOmmU1JgwQu-TE7w2LAa6ihU-zSSoqcXwLQpgeuUk29efs4d4wMWd6Ci63HzZyjy8A', 'Match_ID': 'NA1_3250295301', 'Placement': 4, 'Champions': "[('RekSai', 1, 2, []), ('KogMaw', 0, 3, [57, 12, 26]), ('Twitch', 3, 1, []), ('Ashe', 3, 2, [1, 67, 56]), ('Nocturne', 2, 2, []), ('Skarner', 1, 2, [59]), ('Singed', 4, 1, [10204, 3])]", 'Traits': "[('Alchemist', 1), ('Crystal', 1), ('Metal', 1), ('Poison', 1), ('Predator', 1), ('Set2_Assassin', 

{'PUUID': 'zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMlDTxkQbJzjvt0gGD-d2XOkwS-hNi4-BFQ', 'Match_ID': 'NA1_3250068674', 'Placement': 7, 'Champions': "[('Renekton', 0, 1, []), ('Khazix', 3, 1, []), ('DrMundo', 2, 2, []), ('Sion', 2, 2, []), ('Volibear', 1, 2, []), ('Olaf', 3, 2, []), ('Jax', 1, 2, [])]", 'Traits': '[]'}
{'PUUID': 'zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMlDTxkQbJzjvt0gGD-d2XOkwS-hNi4-BFQ', 'Match_ID': 'NA1_3249990708', 'Placement': 3, 'Champions': "[('Janna', 3, 1, [88]), ('Khazix', 3, 2, []), ('MasterYi', 4, 2, []), ('Lux', 5, 1, []), ('Nocturne', 2, 2, [28, 19, 11]), ('Yasuo', 1, 1, []), ('Qiyana', 2, 2, [37, 33, 10204]), ('Sivir', 2, 2, [24, 10204, 46]), ('RekSai', 1, 2, [])]", 'Traits': "[('Avatar', 1), ('Desert', 1), ('Metal', 1), ('Mystic', 1), ('Predator', 0), ('Set2_Assassin', 1), ('Set2_Blademaster', 2), ('Shadow', 0), ('Wind', 3)]"}
{'PUUID': 'zqelCdkKSSNGoCJA9msW7sP-wmzIW0HPG8b8ElUMwLCIMlDTxkQbJzjvt0gGD-d2XOkwS-hNi4-BFQ', 'Match_ID': 'NA1_3249943171', '

{'PUUID': 'Wnwf5-10yZz4KMzWFXYf4F3y0um3PLAqYwFhVg3lJe0IkoP8fkuE_BiWW9nNY6IFs38AgEELS6pNPw', 'Match_ID': 'NA1_3250396674', 'Placement': 6, 'Champions': "[('Brand', 3, 1, []), ('Qiyana', 2, 2, [10203, 47]), ('Syndra', 1, 2, []), ('Nautilus', 2, 1, []), ('Yorick', 3, 2, [25, 69, 15]), ('Malphite', 3, 1, [66]), ('Thresh', 1, 1, []), ('Taliyah', 0, 2, [10203, 39])]", 'Traits': "[('Inferno', 0), ('Light', 0), ('Mage', 1), ('Mountain', 1), ('Ocean', 2), ('Set2_Assassin', 0), ('Summoner', 0), ('Warden', 1)]"}
{'PUUID': 'Wnwf5-10yZz4KMzWFXYf4F3y0um3PLAqYwFhVg3lJe0IkoP8fkuE_BiWW9nNY6IFs38AgEELS6pNPw', 'Match_ID': 'NA1_3250403091', 'Placement': 4, 'Champions': "[('Soraka', 2, 2, [34]), ('Kindred', 2, 2, [24, 66, 25]), ('Varus', 1, 2, []), ('Janna', 3, 1, []), ('MasterYi', 4, 1, [13]), ('Qiyana', 2, 2, [10203, 15, 38]), ('Nautilus', 2, 2, [10203]), ('', 4, 1, [9])]", 'Traits': "[('Inferno', 1), ('Light', 0), ('Mystic', 1), ('Ocean', 1), ('Set2_Assassin', 0), ('Set2_Blademaster', 0), ('Set2_Ranger'

{'PUUID': 'MmQ9S18WFQcAwNH1zAFuoiJeTKj9S0KYe6dl6Fe2YVuQkMvsNX3uHKYCM97gqlic-rHCiZE5czr4sQ', 'Match_ID': 'NA1_3249778912', 'Placement': 2, 'Champions': "[('Nami', 4, 1, []), ('Zed', 4, 2, [25, 56, 15]), ('Zyra', 0, 2, [35]), ('Annie', 3, 2, []), ('Qiyana', 2, 2, [99]), ('Kindred', 2, 2, [10201, 58]), ('', 4, 1, [7, 46, 78]), ('Thresh', 1, 2, []), ('Diana', 0, 2, [10201])]", 'Traits': "[('Electric', 0), ('Inferno', 2), ('Mystic', 0), ('Ocean', 1), ('Set2_Assassin', 1), ('Set2_Glacial', 0), ('Set2_Ranger', 0), ('Shadow', 0), ('Summoner', 1), ('Warden', 1)]"}
[{'PUUID': 'OkaQSLZuy-FUjgfc_a770gWAvOtn5WLYcBN7toyVLJsc1djhBqxsj360DcaldQDCukF_G9PButnSFg', 'Match_ID': 'NA1_3249805878', 'Placement': 2, 'Champions': "[('Annie', 3, 2, []), ('Zed', 4, 2, [24, 66, 16]), ('Lux', 5, 2, [22, 35, 35]), ('Yorick', 3, 2, []), ('Malphite', 3, 2, [37]), ('Brand', 3, 1, [10201]), ('Nautilus', 2, 2, []), ('Ornn', 0, 1, [10201]), ('', 4, 2, [14, 2])]", 'Traits': "[('Avatar', 1), ('Electric', 3), ('Inferno', 1),

2000

In [342]:
master_sheet_copy = master_sheet

In [330]:
complete_match_data.iloc[0]

puuid                      HbYsPpRS3X2RaQ8n5Sm9n2gKU80f927PzBk_krw4kdyRG6...
match_id                                                      NA1_3249805878
game_length                                                          1959.95
gold_left                                                                  5
last_round                                                                31
level                                                                      8
placement                                                                  4
players_eliminated                                                         0
time_eliminated                                                       1714.1
total_damage_to_players                                                   95
traits                     [('Berserker', 2), ('Desert', 0), ('Electric',...
units                      [('Renekton', 0, 2, []), ('DrMundo', 2, 2, [])...
Name: 0, dtype: object